In [1]:
import warnings
warnings.filterwarnings('ignore')
from  Model_test import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pandas_profiling
from sklearn.decomposition import PCA

In [2]:
pd.options.display.max_columns = 90

In [3]:
df = pd.read_csv('dataset_diabetes/diabetic_data.csv')

In [4]:
df.head(25)

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,?,?,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,?,?,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO
5,35754,82637451,Caucasian,Male,[50-60),?,2,1,2,3,?,?,31,6,16,0,0,0,414,411,250,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30
6,55842,84259809,Caucasian,Male,[60-70),?,3,1,2,4,?,?,70,1,21,0,0,0,414,411,V45,7,None,None,Steady,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO
7,63768,114882984,Caucasian,Male,[70-80),?,1,1,7,5,?,?,73,0,12,0,0,0,428,492,250,8,None,None,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,>30
8,12522,48330783,Caucasian,Female,[80-90),?,2,1,4,13,?,?,68,2,28,0,0,0,398,427,38,8,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO
9,15738,63555939,Caucasian,Female,[90-100),?,3,3,4,12,?,InternalMedicine,33,3,18,0,0,0,434,198,486,8,None,None,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [ ]:
# pandas_profiling.ProfileReport(df)

## Cleaning some data


In [5]:
# Drop becuase all values are the same
df.drop(columns = ['patient_nbr','citoglipton','weight','examide','encounter_id'],inplace=True)

# weight, drop coz 97% missing
# citoglipton, and examide are constant values
# encounter_id is just an id

In [6]:
# race, missing set to other 2%
df.race.replace('?', 'Other',inplace=True)


In [7]:
df.drop(columns = ['diag_1', 'diag_2','diag_3','payer_code'],inplace=True)


In [ ]:
# convert diabetes_med from Yes or No to 1 or 0
# convert readmission from No, <30, >30 to 0, 1, 2


In [8]:
# convert change from No, Ch to 0, 1
df.change = np.where(df.change=='No',0,1)

In [9]:
# convert diabetes_med from Yes or No to 1 or 0
df.diabetesMed = np.where(df.diabetesMed=='Yes',1,0)

In [10]:
df.drop('medical_specialty', axis=1, inplace=True)
# 49,000 are ?

In [11]:
df.max_glu_serum.unique()

array(['None', '>300', 'Norm', '>200'], dtype=object)

In [12]:
df.readmitted = np.where(df.readmitted=='NO', 0, df.readmitted )
df.readmitted = np.where(df.readmitted=='<30', 1,  df.readmitted )
df.readmitted = np.where(df.readmitted=='>30', 2,  df.readmitted )

In [14]:
dummy_df = pd.get_dummies(df,drop_first=True)

In [15]:
dummy_df.drop(columns=['readmitted_1', 'readmitted_2','gender_Unknown/Invalid'],
              inplace=True)

In [16]:
X = dummy_df.copy()
y = df.readmitted.astype(int)

In [17]:
col_names = X.columns
col_names_list = []
for i, col_name in enumerate(col_names):
    c1 = col_name.replace('[','')
    c1 = c1.replace('>','')
    col_names_list.append(c1)
X.columns = col_names_list
    

In [18]:
# Save X, y to a file for faster loading

## Train Test Split

In [19]:
X_train, X_test, y_train, y_test = scale_train_test_split(X,y)

In [20]:
X.shape

(101766, 81)

In [21]:
from sklearn.decomposition import PCA
import numpy as np

pca_1 = PCA(n_components=20)
pca_2 = PCA(n_components=40)
pca_3 = PCA(n_components=60)

principalComponents = pca_1.fit_transform(X)
principalComponents = pca_2.fit_transform(X)
principalComponents = pca_3.fit_transform(X)

print(np.sum(pca_1.explained_variance_ratio_))
print(np.sum(pca_2.explained_variance_ratio_))
print(np.sum(pca_3.explained_variance_ratio_))



# pca = PCA(n_components=27)
# principalComponents = pca.fit_transform(feat)
# print(np.sum(pca.explained_variance_ratio_))

0.9975261363389396
0.9998231122979772
0.9999964313060076


In [22]:
print(pca_1.explained_variance_ratio_)
# print(np.sum(pca_2.explained_variance_ratio_))
# print(np.sum(pca_3.explained_variance_ratio_))

[7.53308472e-01 1.21023440e-01 5.24617478e-02 3.14699116e-02
 1.21953166e-02 6.74375432e-03 4.84606707e-03 3.56312148e-03
 3.24163689e-03 2.74319165e-03 1.42798277e-03 9.91572195e-04
 6.52638602e-04 4.80596162e-04 4.65231385e-04 4.25295470e-04
 4.01577220e-04 3.84210003e-04 3.72210248e-04 3.28162159e-04]


In [23]:
X.head(2)

,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,change,diabetesMed,race_Asian,race_Caucasian,race_Hispanic,race_Other,gender_Male,age_10-20),age_20-30),age_30-40),age_40-50),age_50-60),age_60-70),age_70-80),age_80-90),age_90-100),max_glu_serum_300,max_glu_serum_None,max_glu_serum_Norm,A1Cresult_8,A1Cresult_None,A1Cresult_Norm,metformin_No,metformin_Steady,metformin_Up,repaglinide_No,repaglinide_Steady,repaglinide_Up,nateglinide_No,nateglinide_Steady,nateglinide_Up,chlorpropamide_No,chlorpropamide_Steady,chlorpropamide_Up,glimepiride_No,glimepiride_Steady,glimepiride_Up,acetohexamide_Steady,glipizide_No,glipizide_Steady,glipizide_Up,glyburide_No,glyburide_Steady,glyburide_Up,tolbutamide_Steady,pioglitazone_No,pioglitazone_Steady,pioglitazone_Up,rosiglitazone_No,rosiglitazone_Steady,rosiglitazone_Up,acarbose_No,acarbose_Steady,acarbose_Up,miglitol_No,miglitol_Steady,miglitol_Up,troglitazone_Steady,tolazamide_Steady,tolazamide_Up,insulin_No,insulin_Steady,insulin_Up,glyburide-metformin_No,glyburide-metformin_Steady,glyburide-metformin_Up,glipizide-metformin_Steady,glimepiride-pioglitazone_Steady,metformin-rosiglitazone_Steady,metformin-pioglitazone_Steady
0,6,25,1,1,41,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
1,1,1,7,3,59,0,18,0,0,0,9,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0


In [ ]:
# import xgboost
# xgb_clf = xgb.XGBClassifier()
# xgb_clf.fit(X_train, y_train)


In [24]:
models, model_names = train_model(X_train, y_train)

In [25]:
y_train_predicts = predict_all(X_train, y_train, models)

In [26]:
y_test_predicts = predict_all(X_test, y_test, models)

In [27]:

print('--- Train data ---')
for i,y_train_pred in enumerate(y_train_predicts):
    display_acc_and_f1_score(y_train, y_train_pred, model_names[i])
print('--- Test data ---')
for i,y_test_pred in enumerate(y_test_predicts):
    display_acc_and_f1_score(y_test, y_test_pred, model_names[i])

--- Train data ---
Model: AdaBoost
Accuracy: 0.5809443949478539
F1-Score: 0.5809443949478539
Model: GradientBoost
Accuracy: 0.5910853728840207
F1-Score: 0.5910853728840207
Model: XGBoost
Accuracy: 0.587796761176039
F1-Score: 0.587796761176039
Model: Logistic Regresion
Accuracy: 0.5713143965200985
F1-Score: 0.5713143965200985
--- Test data ---
Model: AdaBoost
Accuracy: 0.5818331892146844
F1-Score: 0.5818331892146844
Model: GradientBoost
Accuracy: 0.5862353588554359
F1-Score: 0.5862353588554359
Model: XGBoost
Accuracy: 0.5840735791211382
F1-Score: 0.5840735791211382
Model: Logistic Regresion
Accuracy: 0.569530697272227
F1-Score: 0.569530697272227


## Base Model

## Columns to inspect

    - Admission_type_id
    - discharge_disposition_id
    - admission_source_id
    - number_of_lab_procedures
    